## Demande de visualisations complémentaires de Thierry du 26/05 :

1. Il faudrait avoir un graphique sur les exportations totales d’eau de vie sur la France entière et en particulier sur les généralités de Bordeaux, Bayonne et Nantes. Il y aussi à partir du milieu des années 20 une augmentation des exportations à partir des ports méridionaux, en particulier Sète. On pourrait avoir des coupes en 1750, une autre année (?) et 1789

2. Autre idée de visualisation: en 1789, regarder les origines des exports. Il est possible qu’une partie des exports de la région de Cognac passe par Bordeaux, l’Armagnac a priori passe par Bayonne (selon Cullen 1998 qui est LA référence sur ce thème). Il y aussi des choses intéressantes dans l’article de Thierry (2013) sur Tonneau-Charente et Rochefort. Il faut distinguer aussi entre “eau de vie simple”  et “eau de vie double” dans TOFLIT18, la 2e catégorie correspond je pense à la meilleure qualité (elle est plus chère et on la trouve moins souvent), donc le Cognac (et Armagnac) 

#### Plan d'action pour viz 1 : FAIT
- 1.1 trend de 1750 à 1789 sur l'évolution de l'export de l'EDV avec données Toflit : 
    * pour la direction de Bayonne
    * pour la direction de Bordeaux
    * pour la direction de Nantes
    * pour les directions restantes
- 1.2 tranches en 1750 et en 1789 : répartition des directions des Fermes exportatrices
    * zoom sur bureaux de Bordeaux, Bayonne et Nantes : vers quels partenaires se destinent les exports d'EDV depuis ces directions en 1750 et en 1789 (regarder aussi au niveau des directions des fermes ??)

#### Plan d'action pour viz 2 : À FAIRE
- repartition origine des exports en 89 (par province, à voir si besoin agréger province en catégories régionales pertinentes plus tard) 
    * zoom sur bureaux de Bordeaux, Bayonne, Tonnay-Charente, Rochefort (La Rochelle ? Nantes ?)(regarder aussi au niveau des directions des fermes ??)
    * pouvoir distinguer "eau-de-vie simple" "eau de vie double" (répartition colorée sur mon bar chart)
 
#### Précision sur les données utilisées : Toflit 
* produits regardés : product_revolutionempire = 'Eaux-de-vie et liqueurs' (en réalité on pourrait affiner en enlevant quelques produits dans cette catégorie, et en ajoutant les produits 'vin et eau-de-vie' et 'vin et eau-de-vie' de la catégorie RE 'Boissons diverses et mélangées')

=> pour la viz 2 j'aurais notamment besoin de préciser les catégories pertinentes grâce à ce tableur : https://docs.google.com/spreadsheets/d/1dD-vLa6ArfBpCi4K-VY8yUVu0IL-ruhX40j9l88GDsg/edit?usp=sharing

les cases oranges sont des menus déroulant dans lesquelles il faudrait spécifier si, dans le cadre de cette étude, le produit à gauche est à considérer comme de l'eau-de-vie simple / double / autre type d'eau-de-vie / pas de l'eau-de-vie

In [ ]:
from poitousprint import Toflit, Portic, get_pointcalls_commodity_purposes_as_toflit_product
from vega import VegaLite
import pandas as pd

toflit_client = Toflit()
portic_client = Portic()

In [ ]:
flows_national_export_edv = toflit_client.get_flows(
    start_year = 1750,
    end_year = 1789,
    export_import = 'Exports',
    product_revolutionempire = 'Eaux-de-vie et liqueurs')

In [ ]:
print("Nombre d'exports d'eau de vie au niveau national entre 1750 et 1789  :", len(flows_national_export_edv))

### 1.1 Évolution des exports d'EDV de 1750 à 1789 : on regarde valeur cumulée sur toute la France

In [ ]:
years_list = [y + 1712 for y in range(120)] # à changer je pense
years = {}

for y in years_list:
    years[str(y)] = {
        "year": y,
        "total_france_export":0,
        "bayonne_export": 0,
        "bordeaux_export": 0,
        "nantes_export": 0,
        "la_rochelle_export":0,
        "marseille_export":0,
        "remaining_france_export":0
    }


for f in flows_national_export_edv: 
    year = str(f['year'].split('.')[0])
    value = float(f['value']) if f['value'] != '' else 0
    years[year]['total_france_export'] += value  
    if f['customs_region'] == 'Bayonne':
        years[year]['bayonne_export'] += value
    elif f['customs_region'] == 'Bordeaux':
        years[year]['bordeaux_export'] += value
    elif f['customs_region'] == 'Nantes':
        years[year]['nantes_export'] += value
    elif f['customs_region'] == 'La Rochelle':
        years[year]['la_rochelle_export'] += value
    elif f['customs_region'] == 'Marseille':
        years[year]['marseille_export'] += value
    else:
        years[year]['remaining_france_export'] += value

part_by_year = []
for year, values in years.items():
    """
    part_by_year.append({
        "year": year,
        "type": "ensemble de la France,
        ",
        "portion": values['total_france_export'] 
    })"""
    part_by_year.append({
        "year": year,
        "type": "Autres directions",
        "portion": values['remaining_france_export'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_year.append({
        "year": year,
        "type": "La Rochelle",
        "portion": values['la_rochelle_export'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_year.append({
        "year": year,
        "type": "Bayonne",
        "portion": values['bayonne_export'] # je ne sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_year.append({
        "year": year,
        "type": "Bordeaux",
        "portion": values['bordeaux_export'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_year.append({
        "year": year,
        "type": "Nantes",
        "portion": values['nantes_export'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_year.append({
        "year": year,
        "type": "Marseille",
        "portion": values['marseille_export'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    
   
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Évolution des exports d'eau-de-vie de 1750 à 1789",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "width": 800, "height": 600,
    "encoding": {
        "x": {
            "field": "year",
            "type": "ordinal", 
            "title": "année"
        },
        "y": {
            "type": "quantitative",
            "field": "portion",
            "title": "Valeur cumulée des exports, en livre tournois"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Direction des Fermes",
            "scale": {
                 "range": ["orange", "pink", "green", "blue", "black", "violet"]
            }
        }
    }
}, pd.DataFrame([y for y in part_by_year if int(y['year']) > 1749 and int(y['year']) < 1790]))


### Qui exporte de l'eau de vie ? 1ere itération en 1750

In [ ]:
flows_1750_export_edv = toflit_client.get_flows(
    year = 1750,
    export_import = 'Exports',
    product_revolutionempire = 'Eaux-de-vie et liqueurs')

print(len(flows_1750_export_edv)," flux d'export d'eau-de-vie trouvés en 1750")

### Qui exporte de l'eau de vie ? 2e itération en 1789

In [ ]:
flows_1789_export_edv = toflit_client.get_flows(
    year = 1789,
    export_import = 'Exports',
    product_revolutionempire = 'Eaux-de-vie et liqueurs')
print(len(flows_1789_export_edv)," flux d'export d'eau-de-vie trouvés en 1789")

### 1.2 Tranches en 1750 et en 1789 : regarder plus précisément les destinataires de ces exports ? 

In [ ]:
partners_set = set() 
for f in flows_national_export_edv:
    partners_set.add(f['partner_grouping'])
# print(len(partners_set), "partenaires commerciaux trouvés\n")
partners = {}

for p in partners_set:
    partners[p] = {
        "partner_grouping": p,
        "exports_from_all_france":0,
        "exports_from_la_rochelle":0,
        "exports_from_bayonne": 0,
        "exports_from_bordeaux": 0,
        "exports_from_nantes": 0,
        "exports_from_marseille":0,
        "exports_from_rest_of_france":0,
    }


for f in flows_1750_export_edv: 
    partner = f['partner_grouping']
    value = float(f['value']) if f['value'] != '' else 0
    partners[partner]['exports_from_all_france'] += value  
    if f['customs_region'] == 'Bayonne':
        partners[partner]['exports_from_bayonne'] += value
    elif f['customs_region'] == 'Bordeaux':
        partners[partner]['exports_from_bordeaux'] += value
    elif f['customs_region'] == 'Nantes':
        partners[partner]['exports_from_nantes'] += value
    elif f['customs_region'] == 'La Rochelle':
        partners[partner]['exports_from_la_rochelle'] += value
    elif f['customs_region'] == 'Marseille':
        partners[partner]['exports_from_marseille'] += value
    else:
        partners[partner]['exports_from_rest_of_france'] += value

part_by_partner = []
for partner, values in partners.items():
    """
    part_by_partner.append({
        "partner": partner,
        "type": "ensemble de la France,
        ",
        "portion": values['total_france_export'] 
    })"""
    part_by_partner.append({
        "partner": partner,
        "type": "La Rochelle",
        "portion": values['exports_from_la_rochelle'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Autres directions",
        "portion": values['exports_from_rest_of_france'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Bayonne",
        "portion": values['exports_from_bayonne'] # je ne sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Bordeaux",
        "portion": values['exports_from_bordeaux'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Nantes",
        "portion": values['exports_from_nantes'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Marseille",
        "portion": values['exports_from_marseille'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    
   
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Destination des exports d'eau-de-vie en 1750",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "width": 600, "height": 400,
    "encoding": {
        "x": {
            "field": "partner",
            "type": "ordinal", 
            "title": "partenaire commercial"
        },
        "y": {
            "type": "quantitative",
            "field": "portion",
            "title": "Valeur cumulée des exports, en livre tournois"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Directions des Fermes exportatrices",
            "scale": {
                 "range": ["orange", "pink", "green", "blue", "black", "violet"]
            }
        }
    }
}, pd.DataFrame([p for p in part_by_partner]))


In [ ]:
partners = {}

for p in partners_set:
    partners[p] = {
        "partner_grouping": p,
        "exports_from_all_france":0,
        "exports_from_la_rochelle":0,
        "exports_from_bayonne": 0,
        "exports_from_bordeaux": 0,
        "exports_from_nantes": 0,
        "exports_from_marseille":0,
        "exports_from_rest_of_france":0,
    }


for f in flows_1789_export_edv: 
    partner = f['partner_grouping']
    value = float(f['value']) if f['value'] != '' else 0
    partners[partner]['exports_from_all_france'] += value  
    if f['customs_region'] == 'Bayonne':
        partners[partner]['exports_from_bayonne'] += value
    elif f['customs_region'] == 'Bordeaux':
        partners[partner]['exports_from_bordeaux'] += value
    elif f['customs_region'] == 'Nantes':
        partners[partner]['exports_from_nantes'] += value
    elif f['customs_region'] == 'La Rochelle':
        partners[partner]['exports_from_la_rochelle'] += value
    elif f['customs_region'] == 'Marseille':
        partners[partner]['exports_from_marseille'] += value
    else:
        partners[partner]['exports_from_rest_of_france'] += value

part_by_partner = []
for partner, values in partners.items():
    """
    part_by_partner.append({
        "partner": partner,
        "type": "ensemble de la France,
        ",
        "portion": values['total_france_export'] 
    })"""
    part_by_partner.append({
        "partner": partner,
        "type": "La Rochelle",
        "portion": values['exports_from_la_rochelle'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Autres directions",
        "portion": values['exports_from_rest_of_france'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Bayonne",
        "portion": values['exports_from_bayonne'] # je ne sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Bordeaux",
        "portion": values['exports_from_bordeaux'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Nantes",
        "portion": values['exports_from_nantes'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    part_by_partner.append({
        "partner": partner,
        "type": "Marseille",
        "portion": values['exports_from_marseille'] # je sais pas si c'est portion vu qu'on a une valeur absolue donnée en livres
    })
    
   
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Destination des exports d'eau-de-vie en 1789",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "width": 600, "height": 500,
    "encoding": {
        "x": {
            "field": "partner",
            "type": "ordinal", 
            "title": "partenaire commercial"
        },
        "y": {
            "type": "quantitative",
            "field": "portion",
            "title": "Valeur cumulée des exports, en livre tournois"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Directions des Fermes exportatrices",
            "scale": {
                 "range": ["orange", "pink", "green", "blue", "black", "violet"]
            }
        }
    }
}, pd.DataFrame([p for p in part_by_partner]))
